In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import os
import re
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
from collections import Counter

DATA_DIR = "/content/drive/MyDrive/Data/红楼梦txt"
OUTPUT_DIR = "/content/hlm_network_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

client = OpenAI(
    api_key="sk-159281b95066415496f0945c798b1179",
    base_url="https://api.deepseek.com/v1"
)

role_dict = {
    "賈寶玉": ["寶玉","寶二爺","寶哥哥","怡紅公子","寶兄弟","玉兄弟"],
    "林黛玉": ["黛玉","林姑娘","颦兒","顰兒","林妹妹","林姐兒"],
    "薛寶釵": ["寶釵","薛姑娘","釵姐兒","寶姐姐"],

    "襲人": ["花襲人","襲人姑娘"],
    "晴雯": ["晴雯姑娘"],
    "麝月": ["麝月姑娘"],
    "紫鵑": ["紫鵑姑娘"],

    "探春": ["三姑娘","探姑娘"],
    "惜春": ["四姑娘"],
    "迎春": ["二姑娘"],

    "賈母": ["老太君","老祖宗"],
    "賈政": ["政老爺","賈政老爺"],
    "王夫人": ["太太","王太太","夫人"],

    "王熙鳳": ["鳳姐","鳳辣子","熙鳳","鳳姑娘"],
    "史湘雲": ["湘雲","史姑娘","雲妹妹"],

    "賈璉": ["璉二爺","璉兄弟"],
    "薛蟠": ["薛大爺","薛大哥"],
}

alias_map = {}
for k,v in role_dict.items():
    for alias in v:
        alias_map[alias] = k
    alias_map[k] = k

def load_chapters_by_phase(phase):
    """根据阶段加载对应的章节"""
    if phase == 1:
        target_files = [f"ch{str(19+i).zfill(3)}.txt" for i in range(5)]
    elif phase == 2:
        target_files = [f"ch{str(26+i).zfill(3)}.txt" for i in range(4)]
    elif phase == 3:
        target_files = [f"ch{str(32+i).zfill(3)}.txt" for i in range(5)]
    else:
        return []

    chapters = []
    for f in target_files:
        file_path = os.path.join(DATA_DIR, f)
        if os.path.exists(file_path):
            with open(file_path, "r", encoding="utf-8") as fp:
                txt = fp.read()
            sentences = re.split(r"[。！？，；]", txt)
            sentences = [s.strip() for s in sentences if s.strip()]
            chapters.append({"file": f, "sentences": sentences})
        else:
            print(f"警告：文件 {f} 不存在")

    return chapters

def extract_roles(text):
    """在一句话里找出现过哪些角色或别名"""
    found = set()
    for alias, canonical in alias_map.items():
        if alias in text:
            found.add(canonical)
    return list(found)

def llm_judge_interaction(s1, roles):
    """
    改进版的互动判断 - 专门针对繁体中文优化
    """
    if len(roles) < 2:
        return []

    prompt = f"""
請仔細分析以下《紅樓夢》繁體中文文本片段：

「{s1}」

**任務要求：**
1. 只識別有明確直接互動的角色對
2. 直接互動包括：直接對話、身體接觸、眼神交流、明確的情感表達、共同參與同一事件
3. 排除：間接提及、旁觀、心理活動、回憶中的互動、單純的稱呼

**紅樓夢特點注意：**
- 注意人物稱謂的變化（如"寶二爺"指賈寶玉、"顰兒"指林黛玉）
- 嚴格區分直接互動和間接提及
- 丫鬟與主子的互動要明確是服務行為還是情感交流
- 注意繁體字詞的細微差異

**當前出現的角色列表（已統一為標準名稱）：** {roles}

請嚴格按照以下格式輸出，只包含確實有直接互動的角色對：
[["角色A","角色B"],["角色C","角色D"]]
如果沒有任何直接互動，請輸出：[]
請務必確保角色名稱與上述列表中的標準名稱完全一致。
"""

    try:
        resp = client.chat.completions.create(
            model="deepseek-chat",
            messages=[{"role":"user","content": prompt}],
            temperature=0.1,
            max_tokens=500
        )
        out = resp.choices[0].message.content.strip()
        print(f"LLM原始輸出: {out}")

        if out.startswith('[') and out.endswith(']'):
            pairs = eval(out)
            valid_pairs = []
            for p in pairs:
                if len(p) == 2 and p[0] in roles and p[1] in roles and p[0] != p[1]:
                    sorted_pair = tuple(sorted([p[0], p[1]]))
                    valid_pairs.append(sorted_pair)
            return valid_pairs
        else:
            print(f"LLM輸出格式不正確: {out}")
            return []
    except Exception as e:
        print(f"LLM解析錯誤: {e}, 輸出: {out}")
        return []

def validate_and_clean_edges(edges, phase):
    """
    基于红楼梦知识验证和清理边数据
    """
    phase_character_focus = {
        1: ["賈寶玉", "林黛玉", "薛寶釵", "襲人", "晴雯", "賈母", "王夫人", "探春", "惜春", "迎春"],
        2: ["賈寶玉", "林黛玉", "薛寶釵", "王熙鳳", "史湘雲", "襲人", "賈母", "王夫人"],
        3: ["賈寶玉", "林黛玉", "薛寶釵", "襲人", "賈政", "王夫人", "賈璉", "王熙鳳", "麝月"]
    }

    valid_edges = []
    focus_chars = phase_character_focus.get(phase, [])

    for a, b in edges:
        if a in focus_chars and b in focus_chars:
            valid_edges.append((a, b))
        elif "賈寶玉" in [a, b] and (a in focus_chars or b in focus_chars):
            valid_edges.append((a, b))
        else:
            print(f"阶段{phase}发现可能不合理的边: {a} - {b}")
            valid_edges.append((a, b))

    return valid_edges

phase_names = {
    1: "天真少年(19-23回)",
    2: "情感觉醒(26-29回)",
    3: "现实冲击(32-36回)"
}

for phase in [1, 2, 3]:
    print(f"\n正在处理阶段{phase}: {phase_names[phase]}")

    chapters = load_chapters_by_phase(phase)
    print(f"阶段{phase}加载了 {len(chapters)} 个章节")

    if len(chapters) == 0:
        print(f"阶段{phase}没有找到任何文件，跳过")
        continue

    edges = []
    WINDOW = 3

    for ch in tqdm(chapters, desc=f"阶段{phase}"):
        sents = ch["sentences"]
        for i in range(len(sents)-WINDOW+1):
            window_text = "。".join(sents[i:i+WINDOW])
            roles_here = extract_roles(window_text)
            if len(roles_here) < 2:
                continue

            pairs = llm_judge_interaction(window_text, roles_here)
            for a,b in pairs:
                edges.append((a,b))

    edges = validate_and_clean_edges(edges, phase)
    print(f"阶段{phase}验证后边数量:", len(edges))

    edge_count = Counter()
    for a,b in edges:
        if a != b:
            key = tuple(sorted([a,b]))
            edge_count[key] += 1

    edges_df = pd.DataFrame(
        [{"source": k[0], "target": k[1], "weight": v} for k,v in edge_count.items()]
    )
    edges_file = os.path.join(OUTPUT_DIR, f"edges_phase{phase}_{phase_names[phase]}.csv")
    edges_df.to_csv(edges_file, index=False, encoding="utf-8-sig")
    print(f"阶段{phase} edges保存到: {edges_file}")
    print(edges_df.head())

    node_count = Counter()
    for a,b in edges:
        node_count[a] += 1
        node_count[b] += 1

    nodes_df = pd.DataFrame(
        [{"id": k, "count": v} for k,v in node_count.items()]
    )
    nodes_file = os.path.join(OUTPUT_DIR, f"nodes_phase{phase}_{phase_names[phase]}.csv")
    nodes_df.to_csv(nodes_file, index=False, encoding="utf-8-sig")
    print(f"阶段{phase} nodes保存到: {nodes_file}")
    print(nodes_df.head())

print("\n所有阶段处理完成！")


正在处理阶段1: 天真少年(19-23回)
阶段1加载了 5 个章节


阶段1:   0%|          | 0/5 [00:00<?, ?it/s]

LLM原始輸出: [["襲人","賈母"]]
LLM原始輸出: [["襲人","賈母"]]
LLM原始輸出: [["襲人","賈母"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"],["賈寶玉","賈母"]]
LLM原始輸出: [["賈母","襲人"]]
LLM原始輸出: [["賈珍","賈璉"],["賈珍","薛蟠"],["賈璉","薛蟠"]]
LLM原始輸出: [["賈珍","賈璉"],["賈珍","薛蟠"],["賈璉","薛蟠"]]
LLM原始輸出: [["賈珍","賈璉"],["賈珍","薛蟠"],["賈璉","薛蟠"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["襲人","賈寶玉"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["襲人","賈寶玉"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","晴雯"]]
LLM原始輸出: [["賈寶玉","晴雯"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["襲人","賈寶玉"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [

阶段1:  20%|██        | 1/5 [02:43<10:52, 163.17s/it]

LLM原始輸出: [["薛寶釵","賈寶玉"]]
LLM原始輸出: [["賈寶玉","林黛玉"],["賈寶玉","薛寶釵"]]
LLM原始輸出: [["賈寶玉","薛寶釵"]]
LLM原始輸出: [["賈寶玉","林黛玉"]]
LLM原始輸出: [["賈寶玉","林黛玉"]]
LLM原始輸出: [["賈寶玉","林黛玉"]]
LLM原始輸出: [["林黛玉","薛寶釵"]]
LLM原始輸出: [["襲人","林黛玉"]]
LLM原始輸出: [["林黛玉","襲人"]]
LLM原始輸出: [["林黛玉","襲人"]]
LLM原始輸出: [["賈寶玉","薛寶釵"]]
LLM原始輸出: [["賈寶玉","薛寶釵"]]
LLM原始輸出: [["林黛玉","薛寶釵"]]
LLM原始輸出: [["林黛玉","薛寶釵"]]
LLM原始輸出: [["林黛玉","薛寶釵"]]
LLM原始輸出: [["薛寶釵","王熙鳳"],["林黛玉","王熙鳳"]]
LLM原始輸出: [["薛寶釵","林黛玉"],["薛寶釵","王熙鳳"],["林黛玉","王熙鳳"]]
LLM原始輸出: [["薛寶釵","林黛玉"]]
LLM原始輸出: [["襲人","賈寶玉"]]
LLM原始輸出: [["襲人","賈寶玉"]]
LLM原始輸出: [["襲人","王夫人"]]
LLM原始輸出: [["襲人","王夫人"]]
LLM原始輸出: [["賈母","賈寶玉"]]
LLM原始輸出: [["賈母","賈寶玉"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["晴雯","麝月"]]
LLM原始輸出: [["賈寶玉","麝月"]]
LLM原始輸出: [["賈寶玉","麝月"]]
LLM原始輸出: [["賈寶玉","麝月"]]
LLM原始輸出: [["賈寶玉","麝月"]]
LLM原始輸出: [["賈寶玉","麝月"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","晴雯"]]
LLM原始輸出: [["賈寶玉","晴雯"]]
LLM原始輸出: [["賈寶玉", "麝月"]]
LLM原始輸出: [["賈寶玉","麝月"]]
LLM原始輸出: [["賈寶玉","麝月"]]
LLM原始輸出: [["賈寶玉",

...

LLM原始輸出: [["林黛玉","薛寶釵"]]
LLM原始輸出: [["林黛玉","薛寶釵"]]
LLM原始輸出: [["李紈","迎春"],["李紈","探春"],["李紈","惜春"],["迎春","探春"],["迎春","惜春"],["探春","惜春"]]
LLM原始輸出: [["李紈","迎春"],["李紈","探春"],["李紈","惜春"],["迎春","探春"],["迎春","惜春"],["探春","惜春"]]
LLM原始輸出: [["李紈","迎春"],["李紈","探春"],["李紈","惜春"],["迎春","探春"],["迎春","惜春"],["探春","惜春"]]
LLM原始輸出: []
**分析說明：**
- 文本中「心裡自己盤算」表明這是角色的內心活動（心理活動）
- 「他怎麼不來瞧瞧寶玉呢」是心理獨白，不是直接對話
- 雖然提到了「寶玉」（賈寶玉）和「鳳姐兒」（王熙鳳），但都是間接提及，沒有發生直接互動
- 沒有描述任何直接對話、身體接觸、眼神交流或共同參與事件

因此，根據任務要求，此片段中不存在任何明確的直接互動角色對。
LLM輸出格式不正確: []
**分析說明：**
- 文本中「心裡自己盤算」表明這是角色的內心活動（心理活動）
- 「他怎麼不來瞧瞧寶玉呢」是心理獨白，不是直接對話
- 雖然提到了「寶玉」（賈寶玉）和「鳳姐兒」（王熙鳳），但都是間接提及，沒有發生直接互動
- 沒有描述任何直接對話、身體接觸、眼神交流或共同參與事件

因此，根據任務要求，此片段中不存在任何明確的直接互動角色對。
LLM原始輸出: [["賈母","王熙鳳"]]
LLM原始輸出: [["賈母","王熙鳳"]]
LLM原始輸出: [["賈母","王熙鳳"]]
LLM原始輸出: [["紫鵑","薛寶釵"]]
LLM原始輸出: [["紫鵑","薛寶釵"]]
LLM原始輸出: [["林黛玉","紫鵑"]]
LLM原始輸出: [["林黛玉","紫鵑"]]
LLM原始輸出: [["林黛玉","紫鵑"]]
LLM原始輸出: [["林黛玉","紫鵑"]]
LLM原始輸出: [["薛蟠","薛寶釵"]]
LLM原始輸出: [["薛寶釵","薛蟠"]]
LLM原始輸出: [["薛蟠","薛寶釵"]]
LLM原始輸出: [["薛姨媽","薛寶釵"]]
LLM原始輸出: [[

阶段3:  80%|████████  | 4/5 [12:08<03:18, 198.98s/it]

LLM原始輸出: [["賈寶玉","林黛玉"]]
LLM原始輸出: [["賈母","王夫人"]]
LLM原始輸出: [["賈寶玉","賈政"]]
LLM原始輸出: [["賈政","賈寶玉"]]
LLM原始輸出: [["賈母","李嬤嬤"],["賈母","襲人"]]
LLM原始輸出: [["賈母","襲人"]]
LLM原始輸出: [["賈母","李嬤嬤"],["賈母","襲人"],["李嬤嬤","賈寶玉"],["襲人","賈寶玉"]]
LLM原始輸出: [["賈母","王夫人"]]
LLM原始輸出: [["賈母","王夫人"]]
LLM原始輸出: [["賈母","王夫人"]]
LLM原始輸出: [["王熙鳳","金釧兒"]]
LLM原始輸出: [["薛寶釵","王夫人"],["林黛玉","王夫人"]]
LLM原始輸出: [["薛寶釵","王夫人"],["林黛玉","王夫人"]]
LLM原始輸出: [["王熙鳳","王夫人"]]
LLM原始輸出: [["王熙鳳","王夫人"]]
LLM原始輸出: [["王熙凤","王夫人"]]
LLM原始輸出: [["王夫人","王熙鳳"]]
LLM原始輸出: [["王夫人","王熙鳳"]]
LLM原始輸出: [["王夫人","王熙鳳"]]
LLM原始輸出: [["王夫人","王熙鳳"]]
LLM原始輸出: [["王夫人","王熙鳳"]]
LLM原始輸出: [["王夫人","王熙凤"]]
LLM原始輸出: [["王夫人","王熙凤"]]
LLM原始輸出: [["王夫人","王熙鳳"]]
LLM原始輸出: [["王夫人","王熙鳳"]]
LLM原始輸出: [["王熙鳳","襲人"]]
LLM原始輸出: [["襲人","王夫人"]]
LLM原始輸出: [["王夫人","襲人"]]
LLM原始輸出: [["王夫人","襲人"]]
LLM原始輸出: [["王夫人","王熙鳳"]]
LLM原始輸出: [["王熙鳳","襲人"]]
LLM原始輸出: [["王熙鳳","襲人"]]
LLM原始輸出: [["賈寶玉","王夫人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["賈寶玉","襲人"]]
LLM原始輸出: [["王夫人","王熙鳳"]]
LLM原始輸出: [["王夫人

阶段3: 100%|██████████| 5/5 [15:33<00:00, 186.75s/it]

LLM原始輸出: [["賈寶玉","王夫人"]]
阶段3发现可能不合理的边: 史湘雲 - 襲人
阶段3发现可能不合理的边: 史湘雲 - 襲人
阶段3发现可能不合理的边: 史湘雲 - 襲人
阶段3发现可能不合理的边: 史湘雲 - 襲人
阶段3发现可能不合理的边: 史湘雲 - 襲人
阶段3发现可能不合理的边: 史湘雲 - 襲人
阶段3发现可能不合理的边: 史湘雲 - 薛寶釵
阶段3发现可能不合理的边: 史湘雲 - 林黛玉
阶段3发现可能不合理的边: 史湘雲 - 林黛玉
阶段3发现可能不合理的边: 史湘雲 - 襲人
阶段3发现可能不合理的边: 史湘雲 - 襲人
阶段3发现可能不合理的边: 史湘雲 - 襲人
阶段3发现可能不合理的边: 史湘雲 - 襲人
阶段3发现可能不合理的边: 史湘雲 - 襲人
阶段3发现可能不合理的边: 史湘雲 - 林黛玉
阶段3发现可能不合理的边: 史湘雲 - 林黛玉
阶段3发现可能不合理的边: 史湘雲 - 襲人
阶段3发现可能不合理的边: 史湘雲 - 襲人
阶段3发现可能不合理的边: 史湘雲 - 襲人
阶段3发现可能不合理的边: 王夫人 - 賈母
阶段3发现可能不合理的边: 王夫人 - 賈母
阶段3发现可能不合理的边: 賈政 - 賈母
阶段3发现可能不合理的边: 王夫人 - 賈母
阶段3发现可能不合理的边: 王夫人 - 賈母
阶段3发现可能不合理的边: 王夫人 - 賈母
阶段3发现可能不合理的边: 王夫人 - 賈母
阶段3发现可能不合理的边: 賈政 - 賈母
阶段3发现可能不合理的边: 王夫人 - 賈母
阶段3发现可能不合理的边: 王夫人 - 賈母
阶段3发现可能不合理的边: 賈政 - 賈母
阶段3发现可能不合理的边: 賈政 - 賈母
阶段3发现可能不合理的边: 賈政 - 賈母
阶段3发现可能不合理的边: 史湘雲 - 襲人
阶段3发现可能不合理的边: 薛蟠 - 襲人
阶段3发现可能不合理的边: 薛寶釵 - 薛蟠
阶段3发现可能不合理的边: 薛蟠 - 襲人
阶段3发现可能不合理的边: 薛寶釵 - 薛蟠
阶段3发现可能不合理的边: 薛蟠 - 襲人
阶段3发现可能不合理的边: 晴雯 - 襲人
阶段3发现可能不合理的边: 晴雯 - 王夫人
阶段3发现可能不合理的边: 晴雯 - 王夫人
阶段3发现可能不合理的边: 晴雯 - 林黛玉
阶段3发现可能不合理的边: